In [8]:
from langchain_community.tools import DuckDuckGoSearchResults
from langchain_ollama import ChatOllama
from langchain_aws import ChatBedrock
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from dotenv import load_dotenv

load_dotenv()

True

In [13]:
# Setup Tavily Search Tool
# search = TavilySearchResults(max_results=3)
search = DuckDuckGoSearchResults(max_results=5, output_format='list')
# Initialize Language Model
llm = ChatBedrock(model="us.meta.llama3-2-11b-instruct-v1:0", streaming=True)

In [14]:
search_results = search.invoke('Who is the CEO of AMD?')
print(search_results)

[{'snippet': "Which is thanks in large part to Su's leadership. When she became CEO a decade ago, AMD stock was languishing around $3, its share of the data-center chip market had fallen so far that ...", 'title': 'Lisa Su: CEO of the Year 2024 - TIME', 'link': 'https://time.com/7200909/ceo-of-the-year-2024-lisa-su/'}, {'snippet': 'Su, who lives and works in Austin, has headed Advanced Micro Devices, Inc. (AMD) as its CEO since 2014. Over the past decade, AMD surpassed its rival Intel in total company value for the first ...', 'title': "Austin-based AMD CEO named Time's 2024 CEO of the Year - mySA", 'link': 'https://www.mysanantonio.com/business/article/lisa-su-amd-time-ceo-2024-19978092.php'}, {'snippet': "Advanced Micro Devices (AMD) CEO Lisa Su's out-of-hours expectations are going viral after being named Time's CEO of the year.. There are two types of leaders: those who promote a firm clock-off ...", 'title': "Meet Time CEO of the year Lisa Su, who grew AMD's share price by 50x ...

In [54]:
# Extract the results directly from the artifact dictionary
results = search_results.artifact['results']

results

[{'title': "CEO Of The Year: A Conversation With AMD's Revolutionary Lisa Su",
  'url': 'https://chiefexecutive.net/ceo-of-the-year-a-conversation-with-amds-revolutionary-lisa-su/',
  'content': '“Lisa Su’s leadership in transforming AMD into a global leader in innovation and technology is truly remarkable,” says Ed Bastian, CEO of Delta Air Lines and Chief Executive’s 2023 CEO of the Year, who served on this year’s selection committee. The way I like to say it is, “Look, if you’re going to trust me or trust AMD with your most important projects, then I’d like you to know there’s a face behind the company.” So for our largest customers, I want you to trust me.',
  'score': 0.82254606,
  'raw_content': None},
 {'title': 'Dr. Lisa Su - AMD',
  'url': 'https://www.amd.com/en/corporate/leadership/lisa-su.html',
  'content': "We Have an Incredibly Competitive AI Product\xa0–\xa0CNBC\nAMD CEO sees a Golden Opportunity for AMD AI Chips\xa0–\xa0FORTUNE\nAMD CEO Lisa Su: We will start productio

In [4]:
# for chunk in llm.stream("hi"):
#     print(chunk.content, end="", flush=True)
resp = llm.invoke("HI")
print(resp.content)

It's nice to meet you. Is there something I can help you with or would you like to chat?


In [3]:
# Create Prompt Template
prompt = ChatPromptTemplate.from_template("""
Using the following web search results, provide a comprehensive answer to the question.

Search Results:
{search_results}

Question: {question}

Guidelines:
- Base your answer solely on the provided search results
- Be clear and concise
- Include key information from the sources
- Cite sources at the end of your response

Answer:""")

In [4]:

# Create Chain for Web Search and Answer Generation
def tavily_search_and_answer(query):
    # Perform web search
    search_results = search.invoke(query)
    
    # Format search results
    formatted_results = "\n\n".join([
        f"Source {i+1} ({result['url']}):\n{result['content']}" 
        for i, result in enumerate(search_results)
    ])
    
    # Generate answer chain
    chain = (
        prompt 
        | llm 
        | StrOutputParser()
    )
    
    # Invoke chain
    answer = chain.invoke({
        "search_results": formatted_results, 
        "question": query
    })
    
    # Append sources
    full_response = answer + "\n\nSources:\n" + "\n".join([
        f"{i+1}. {result['url']}" 
        for i, result in enumerate(search_results)
    ])
    
    return full_response

In [5]:
result = tavily_search_and_answer("What is the stock price of NVIDIA right now?")
print(result)

Unfortunately, the search results do not provide a real-time stock price of NVIDIA (NVDA) at the moment. However, I can provide some information on how to find the current stock price and some recent price movements.

According to Source 2 (Google Finance), NVIDIA's stock price is not displayed in real-time, but it provides a link to the current price and news. Source 3 (Moneycontrol) and Source 5 (MarketWatch) also provide a live stock price, but it is not visible in the search results.

Source 1 (The Wall Street Journal) and Source 4 (Stockanalysis) do not provide a real-time stock price, but they do provide some historical data and statistics.

Recent price movements:

* According to Source 2, NVIDIA's stock price has been affected by various news articles, including reports of Blackwell AI server issues ahead of earnings.
* Source 3 mentions that the share price is up or down, but does not provide a specific price.

To find the current stock price of NVIDIA, I recommend visiting th

In [67]:
from langchain_core.messages import HumanMessage, AIMessage, SystemMessage

messages = "How are you?"

system_prompt = """You are a Decider. Given a Conversation History or an individual question, your job is to analyze and DECIDE if you need to either output 'CASUAL_CHAT' or 'SEARCH'.
    'SEARCH' - output this if the user asked something that its not in the Conversation History so that the decision is that you need to search information.
    'CHAT': output this whenever the user is talking about something already present in the Conversation History and you wouldnt need to perform any search.
    Your MUST only respond in one of the two fixed words and nothing else: 'SEARCH' or 'CHAT'.
    """
decision_llm = ChatBedrock(model="us.meta.llama3-2-11b-instruct-v1:0")
decision = decision_llm.invoke([
    SystemMessage(content=system_prompt),
    HumanMessage(content=messages)
]).content.strip().upper()
print(decision)

CHAT
